In [1]:
label = [('hotel-area', 10),
    ('hotel-bookday', 11),
    ('restaurant-bookday', 15),
    ('restaurant-bookpeople', 17),
    ('attraction-area', 18),
    ('train-day', 19),
    ('restaurant-area', 23),
    ('restaurant-booktime', 33),
    ('attraction-name', 39),
    ('hotel-name', 64),
    ('restaurant-name', 72)]
label = [k for k,v in label]
label

['hotel-area',
 'hotel-bookday',
 'restaurant-bookday',
 'restaurant-bookpeople',
 'attraction-area',
 'train-day',
 'restaurant-area',
 'restaurant-booktime',
 'attraction-name',
 'hotel-name',
 'restaurant-name']

In [6]:
import jsonlines
import os
import json
from sentence_transformers import SentenceTransformer, util



slot_des = {
    "hotel-pricerange":["price budget of the hotel", "preferred cost of the hotel"],
    "hotel-type": ["what is the type of the hotel", "type of hotel building"],
    "hotel-parking":["parking facility at the hotel", "whether the hotel has parking", "does the hotel have parking"],
    "hotel-bookstay":["length of stay at the hotel", "how many days you want to stay at the hotel for"],
    "hotel-bookday":["day of the hotel booking", "what day of the week you want to start staying at the hotel"],
    "hotel-bookpeople":["number of people for the hotel booking", "how many people are staying at the hotel"],
    "hotel-area":["area or place of the hotel", "rough location of the hotel", "preferred location of the hotel"],
    "hotel-stars":["star rating of the hotel", "rating of the hotel out of five stars"],
    "hotel-internet":["internet option at the hotel", "whether the hotel has internet"],
    "hotel-name":["name of the hotel", "which hotel are you looking for"],
    "train-destination":["destination of the train", "what train station you want to travel to", "destination or drop-off location of the train"],
    "train-day":["day of the train", "what day you want to take the train"],
    "train-departure":["departure location of the train", "what train station you want to leave from"],
    "train-arriveby":["arrival time of the train", "what time you want the train to arrive at your destination station by", "when you want to arrive at your destination by train"],
    "train-bookpeople":["number of people booking for train", "how many people you need train booking for", "how many train tickets you need"],
    "train-leaveat":["leaving time for the train", "what time you want the train to leave your departure station by", "when you want to arrive at your destination by train"],
    "attraction-type":["type of the attraction", "type of attraction or point of interest"],
    "attraction-area":["area or place of the attraction", "area to search for attractions", "preferred location for attraction"],
    "attraction-name":["name of the attraction", "which attraction are you looking for"],
    "restaurant-bookpeople":["number of people booking the restaurant", "how many people for the restaurant reservation"],
    "restaurant-bookday":["day of the restaurant booking", "what day of the week to book the table at the restaurant"],
    "restaurant-booktime":["time of the restaurant booking", "what time to book the table at the restaurant" ],
    "restaurant-food":["food type for the restaurant", "the cuisine of the restaurant you are looking for"],
    "restaurant-pricerange":["price budget for the restaurant", "preferred cost of the restaurant"],
    "restaurant-name":["name of the restaurant", "which restaurant are you looking for"],
    "restaurant-area":["area or place of the restaurant", "preferred location of restaurant"],
    "taxi-leaveat":["leaving time of taxi", "when you want the taxi to pick you up", "what time you want the taxi to leave your departure location by"],
    "taxi-destination":["destination of taxi", "where you want the taxi to drop you off", "what place do you want the taxi to take you to"],
    "taxi-departure":["departure location of taxi", "where you want the taxi to pick you up", "what place do you want to meet the taxi"],
    "taxi-arriveby":["arrival time of taxi", "when you want the taxi to drop you off at your destination", "what time you to arrive at your destination by taxi"],
    "bus-people":["number of people booking bus tickets", "how many people are riding the bus"],
    "bus-leaveAt":["leaving time of bus", "when you want the bus to pick you up", "what time you want the bus to leave your departure location by"],
    "bus-destination":["destination of taxi", "where you want the bus to drop you off", "what place do you want the bus to take you to"],
    "bus-day":["day to use the bus tickets", "what day of the week you want to ride the bus"],
    "bus-arriveBy":["arrival time of bus", "when you want the bus to drop you off at your destination", "what time you to arrive at your destination by bus"],
    "bus-departure":["departure location of bus", "where you want the bus to pick you up", "what place do you want to meet the bus"],
    "hospital-department":["name of hospital department", "type of medical care", "what department of the hospital are you looking for"]
}

slot_des = {k:v[0] for k,v in slot_des.items()}
slot_des2corpus_id = {}
i = 0
for k,v in slot_des.items():
    slot_des2corpus_id[k] = 'corpus_{a}'.format(a=i)
    i = i+1


data_list = []
data_path = 'D:\Python4work\paper_modified_test\gte_ft_data_test.jsonl'
with open(data_path, 'r') as file_obj:
    for line in jsonlines.Reader(file_obj):
        data_list.append(line)
print('测试集总数: ', len(data_list))


refer_dict = {}
for k,v in data_list[0]['referred'].items():
    if v != 'none':
        refer_dict[k] = v

processed_data = []
for i in range(len(data_list)):
    query = [' '.join(data_list[i]['user_utt']), ' '.join(data_list[i]['system_utt'])][::-1]
    refer = {}
    for k,v in data_list[i]['referred'].items():
        if v != 'none':
            refer[k] = v
    processed_data.append({'query': query, 'corpus': refer.copy(), 'dialog_seen': data_list[i]['diag_seen_slots']})



all_data = []
for i in range(len(processed_data)):
    text = ' '.join(processed_data[i]['query'])
    corpus_embedding = {k:slot_des[k] for k,v in processed_data[i]['dialog_seen'].items()}
    #corpus_embedding = {k:model.encode(slot_des[k]) for k,v in processed_data[i]['dialog_seen'].items()}
    for wait4fill, known in processed_data[i]['corpus'].items():
        b = 'which slot has the same value with {slot}'.format(slot=wait4fill)
        #b = 'we need to know {c} and find the slot which has the same value with it.'.format(c=slot_des[wait4fill])
        query_text = ' '.join(processed_data[i]['query']+[b])
        all_data.append([query_text, known, corpus_embedding])

all_data

测试集总数:  270


[["the postcode is cb21tt . are you looking for a taxi from old schools to the golden wok ? yes i do . i ' d like to make sure i arrive at the restaurant by the booked time . can you check ? which slot has the same value with taxi-arriveby",
  'restaurant-booktime',
  {'restaurant-name': 'name of the restaurant',
   'restaurant-bookday': 'day of the restaurant booking',
   'restaurant-booktime': 'time of the restaurant booking',
   'restaurant-bookpeople': 'number of people booking the restaurant',
   'attraction-type': 'type of the attraction',
   'attraction-name': 'name of the attraction',
   'taxi-destination': 'destination of taxi'}],
 ['booking was successful . reference number is : 9hmd04uw . anything else ? i would love to find a restaurant in the same price range as the lovell lodge . which slot has the same value with restaurant-pricerange',
  'hotel-pricerange',
  {'hotel-internet': 'internet option at the hotel',
   'hotel-type': 'what is the type of the hotel',
   'hotel-p

In [7]:
dict1 = {name:[] for name in ['hotel-area',
 'hotel-bookday',
 'restaurant-bookday',
 'restaurant-bookpeople',
 'attraction-area',
 'train-day',
 'restaurant-area',
 'restaurant-booktime',
 'attraction-name',
 'hotel-name',
 'restaurant-name']}
dict1

{'hotel-area': [],
 'hotel-bookday': [],
 'restaurant-bookday': [],
 'restaurant-bookpeople': [],
 'attraction-area': [],
 'train-day': [],
 'restaurant-area': [],
 'restaurant-booktime': [],
 'attraction-name': [],
 'hotel-name': [],
 'restaurant-name': []}

In [8]:
for item in all_data:
    try:
        dict1[item[1]].append((item[0], item[2]))
    except:
        continue


In [9]:
dict1

{'hotel-area': [('as i mentioned it is cherap and the phone number is 01223316074 . okay . now could you help me find a restaurant in the expensive price range that is in the same area as the hotel ? which slot has the same value with restaurant-area',
   {'hotel-area': 'area or place of the hotel',
    'hotel-stars': 'star rating of the hotel'}),
  ("okay , you ' re all set . the hotel is booked and the reference number is q3vvcad8 . can i help you with anything else today ? thank you . i am also looking for a restaurant that serves chinese and that is in the same area as the hotel . which slot has the same value with restaurant-area",
   {'hotel-area': 'area or place of the hotel',
    'hotel-pricerange': 'price budget of the hotel',
    'hotel-stars': 'star rating of the hotel',
    'hotel-type': 'what is the type of the hotel',
    'hotel-bookday': 'day of the hotel booking',
    'hotel-bookpeople': 'number of people for the hotel booking',
    'hotel-bookstay': 'length of stay at 